In [ ]:
%%capture
!pip install fastapi uvicorn onnxruntime torch soundfile numpy aiofiles starlette
!pip install pyngrok
!git clone https://github.com/SparkAudio/Spark-TTS
!pip install omegaconf einx
!pip install aiortc

In [2]:
from pyngrok import ngrok

# Gắn ngrok token của bạn (chạy 1 lần)
ngrok.set_auth_token("2n6gGjwozlbdtvvfyCgJZOJO1o3_jqxBHhEAAnqviQNL9jiQ")


In [6]:
# import numpy as np
# import onnxruntime as ort
# from transformers import AutoTokenizer, AutoModelForCausalLM, AutoProcessor
# import torch
# import string
# import io
# import soundfile as sf
# import base64
# from fastapi import FastAPI, WebSocket
# from starlette.websockets import WebSocketState
# import json
# import re
# import asyncio
# import sys
# import logging
# from huggingface_hub import snapshot_download
# from fastapi.middleware.cors import CORSMiddleware

# # Logging setup
# logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
# logger = logging.getLogger(__name__)

# # Import Spark-TTS
# sys.path.append("Spark-TTS")
# from sparktts.models.audio_tokenizer import BiCodecTokenizer

# # FastAPI app
# app = FastAPI()

# # Enable CORS for all origins (adjust as needed for production)
# app.add_middleware(
#     CORSMiddleware,
#     allow_origins=["*"],  
#     allow_credentials=True,
#     allow_methods=["*"],
#     allow_headers=["*"],
# )

# # Constants
# HG_MODEL = "livekit/turn-detector"
# ONNX_FILENAME = "/kaggle/input/model_2/onnx/default/1/model_quantized.onnx"
# EOU_THRESHOLD = 0.5
# MAX_HISTORY = 2
# MAX_HISTORY_TOKENS = 512
# PUNCS = string.punctuation.replace("'", "")

# # Load models
# turn_tokenizer = AutoTokenizer.from_pretrained(HG_MODEL)
# onnx_session = ort.InferenceSession(ONNX_FILENAME, providers=["CPUExecutionProvider"])
# qwen_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")
# qwen_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct", torch_dtype=torch.float16).to("cuda")

# snapshot_download("beyoru/Spark-TTS-0.5B-with-KafkaSpark", local_dir="Spark-TTS-0.5B-with-KafkaSpark")
# spark_model = AutoModelForCausalLM.from_pretrained(
#     "Spark-TTS-0.5B-with-KafkaSpark/LLM"
# ).to("cuda")
# spark_tokenizer = AutoProcessor.from_pretrained("Spark-TTS-0.5B-with-KafkaSpark/LLM")
# audio_tokenizer = BiCodecTokenizer("Spark-TTS-0.5B-with-KafkaSpark")

# # Helpers
# # Helper functions
# def softmax(logits):
#     try:
#         exp_logits = np.exp(logits - np.max(logits))
#         return exp_logits / np.sum(exp_logits)
#     except Exception as e:
#         logger.error(f"Softmax error: {e}")
#         return np.zeros_like(logits)


# def normalize_text(text):
#     try:

#         def strip_puncs(text):
#             return text.translate(str.maketrans("", "", PUNCS))

#         return " ".join(strip_puncs(text).lower().split())
#     except Exception as e:
#         logger.error(f"Normalize text error: {e}")
#         return text


# def format_chat_ctx(chat_ctx):
#     try:
#         new_chat_ctx = []
#         for msg in chat_ctx:
#             if msg["role"] in ("user", "assistant"):
#                 content = normalize_text(msg["content"])
#                 if content:
#                     msg["content"] = content
#                     new_chat_ctx.append(msg)
#         convo_text = turn_tokenizer.apply_chat_template(
#             new_chat_ctx,
#             add_generation_prompt=False,
#             add_special_tokens=False,
#             tokenize=False,
#         )
#         ix = convo_text.rfind("<|im_end|>")
#         return convo_text[:ix]
#     except Exception as e:
#         logger.error(f"Format chat context error: {e}")
#         return ""


# def calculate_eou(chat_ctx, session):
#     try:
#         formatted_text = format_chat_ctx(chat_ctx[-MAX_HISTORY:])
#         inputs = turn_tokenizer(
#             formatted_text,
#             return_tensors="np",
#             truncation=True,
#             max_length=MAX_HISTORY_TOKENS,
#         )
#         input_ids = inputs["input_ids"].astype(np.int64)
#         outputs = session.run(["logits"], {"input_ids": input_ids})
#         logits = outputs[0][0, -1, :]
#         probs = softmax(logits)
#         eou_token_id = turn_tokenizer.encode("<|im_end|>")[0]
#         logger.debug(f"Input text: {formatted_text}, EOU token ID: {eou_token_id}, Probs: {probs}")
#         return probs[eou_token_id]
#     except Exception as e:
#         logger.error(f"EOU calculation error: {e}")
#         return 0.0

# async def generate_speech_from_text(text: str) -> np.ndarray:
#     try:
#         prompt = "".join(
#             [
#                 "<|task_tts|>",
#                 "<|start_content|>",
#                 text,
#                 "<|end_content|>",
#                 "<|start_global_token|>",
#             ]
#         )
#         model_inputs = spark_tokenizer([prompt], return_tensors="pt").to(
#             spark_model.device
#         )
#         generated_ids = spark_model.generate(
#             **model_inputs,
#             max_new_tokens=1024,
#             temperature = 0.2,
#             do_sample = True,
#             # eos_token_id=spark_tokenizer.eos_token_id,
#             pad_token_id=spark_tokenizer.pad_token_id,
#         )
#         generated_ids_trimmed = generated_ids[:, model_inputs.input_ids.shape[1] :]
#         predicts_text = spark_tokenizer.batch_decode(
#             generated_ids_trimmed, skip_special_tokens=False
#         )[0]
#         semantic_matches = re.findall(r"<\|bicodec_semantic_(\d+)\|>", predicts_text)
#         if not semantic_matches:
#             logger.warning("No semantic matches found in TTS output")
#             return np.array([], dtype=np.float32)
#         pred_semantic_ids = (
#             torch.tensor([int(token) for token in semantic_matches]).long().unsqueeze(0)
#         )
#         global_matches = re.findall(r"<\|bicodec_global_(\d+)\|>", predicts_text)
#         pred_global_ids = (
#             torch.tensor([int(token) for token in global_matches]).long().unsqueeze(0)
#             if global_matches
#             else torch.zeros((1, 1), dtype=torch.long)
#         )
#         pred_global_ids = pred_global_ids.unsqueeze(0)
#         wav_np = audio_tokenizer.detokenize(
#             pred_global_ids.squeeze(0), pred_semantic_ids
#         )
#         return wav_np
#     except Exception as e:
#         logger.error(f"Speech generation error: {e}")
#         return np.array([], dtype=np.float32)

# def waveform_to_base64(waveform: np.ndarray, sample_rate=16000) -> str:
#     try:
#         if waveform.size == 0: 
#             return ""
#         buf = io.BytesIO()
#         sf.write(buf, waveform, sample_rate, format="WAV")
#         return base64.b64encode(buf.getvalue()).decode("utf-8")
#     except Exception as e:
#         logger.error(f"waveform->b64 error: {e}")
#         return ""

# class ConversationManager:
#     def __init__(self):
#         self.chat_history = []
#         self.sample_rate = 16000
#         self.is_processing = False
#         self.lock = asyncio.Lock()
#         self.transcript_buffer = []

#     async def process_conversation(self, transcript: str) -> tuple:
#         async with self.lock:
#             if self.is_processing:
#                 logger.info("Skipping transcript processing: already in progress")
#                 return transcript, 0.0, False, None, None
#             self.is_processing = True
#             try:
#                 if not transcript.strip():
#                     logger.debug("Received empty transcript, skipping")
#                     return transcript, 0.0, False, None, None

#                 # Log input
#                 logger.debug(f"Received transcript: {transcript}")

#                 # Add new transcript to buffer
#                 self.transcript_buffer.append(transcript)
#                 logger.debug(f"Buffered transcript: {transcript}, Buffer: {self.transcript_buffer}")

#                 # Calculate EOU with current chat history plus buffered transcripts
#                 current_transcript = " ".join(self.transcript_buffer)
#                 messages = self.chat_history + [
#                     {"role": "user", "content": current_transcript}
#                 ]
#                 eou_prob = calculate_eou(messages, onnx_session)
#                 logger.debug(f"EOU score for '{current_transcript}': {eou_prob}")

#                 # Set is_final based on EOU score
#                 is_final = eou_prob >= EOU_THRESHOLD
#                 logger.debug(f"is_final set to {is_final} based on eou_score: {eou_prob}")

#                 # Process only if is_final is True (i.e., eou_prob >= EOU_THRESHOLD)
#                 if not is_final:
#                     logger.debug(f"EOU score {eou_prob} below threshold {EOU_THRESHOLD}, buffering")
#                     return current_transcript, eou_prob, False, None, None

#                 # Process the concatenated transcript
#                 logger.info(f"Processing final transcript: {current_transcript}")
#                 inputs = qwen_tokenizer.apply_chat_template(
#                     messages, add_generation_prompt=True, return_tensors="pt"
#                 ).to(qwen_model.device)
#                 outputs = qwen_model.generate(inputs, max_new_tokens=156)
#                 generated_text = qwen_tokenizer.decode(
#                     outputs[0][inputs.shape[1]:], skip_special_tokens=True
#                 )
#                 logger.info(f"Generated response: {generated_text}")

#                 # Update chat history
#                 self.chat_history.append({"role": "user", "content": current_transcript})
#                 self.chat_history.append({"role": "assistant", "content": generated_text})
#                 if len(self.chat_history) > MAX_HISTORY * 2:
#                     self.chat_history = self.chat_history[-MAX_HISTORY * 2:]

#                 # Generate audio
#                 generated_waveform = await generate_speech_from_text(generated_text)
#                 audio_base64 = waveform_to_base64(generated_waveform, self.sample_rate)

#                 # Clear buffer after processing
#                 self.transcript_buffer = []
#                 logger.debug("Cleared transcript buffer")

#                 return current_transcript, eou_prob, True, generated_text, audio_base64
#             except Exception as e:
#                 logger.error(f"Conversation processing error: {e}")
#                 return current_transcript, eou_prob, False, None, None
#             finally:
#                 self.is_processing = False

# @app.get("/")
# async def root():
#     return {"message": "Assistant WebSocket server is running. Connect via /ws/assistant"}

# @app.get("/test")
# async def test_page():
#     return """
#     <!DOCTYPE html>
#     <html>
#     <head><title>Assistant WS Test</title></head>
#     <body>
#     <h1>Assistant WebSocket Test</h1>
#     <textarea id="transcript" rows="4" cols="50"></textarea><br/>
#     <button onclick="sendTranscript()">Send Transcript</button>
#     <pre id="output"></pre>
#     <script>
#       const ws = new WebSocket("ws://" + location.host + "/ws/assistant");
#       ws.onmessage = (event) => {
#         const data = JSON.parse(event.data);
#         document.getElementById('output').textContent += JSON.stringify(data, null, 2) + "\\n";
#       };
#       function sendTranscript() {
#         const transcript = document.getElementById('transcript').value;
#         ws.send(JSON.stringify({transcript: transcript}));
#       }
#     </script>
#     </body>
#     </html>
#     """

# @app.on_event("startup")
# async def startup_event():
#     logger.info("Starting Assistant server...")

# @app.on_event("shutdown")
# async def shutdown_event():
#     logger.info("Shutting down Assistant server...")

# @app.websocket("/ws/assistant")
# async def websocket_endpoint(websocket: WebSocket):
#     await websocket.accept()
#     cm = ConversationManager()
#     try:
#         while True:
#             try:
#                 data = json.loads(await websocket.receive_text())
#                 transcript = data.get("transcript", "")
#                 logger.info(f"WebSocket input: transcript={transcript}")
#                 if not transcript.strip():
#                     continue
#                 # Server quyết định is_final dựa trên eou_score
#                 transcript, eou_score, complete, resp, audio_b64 = await cm.process_conversation(transcript)
#                 await websocket.send_json({"type": "transcript", "transcript": transcript, "eou_score": float(eou_score), "is_complete": complete})
#                 if resp:
#                     await asyncio.sleep(0.1)
#                     await websocket.send_json({"type": "assistant_response", "response": resp, "audio": audio_b64})
#             except Exception as e:
#                 logger.error(f"WebSocket error: {e}")
#                 if websocket.client_state == WebSocketState.CONNECTED:
#                     await websocket.close(code=1000, reason=str(e))
#     except Exception as e:
#         logger.error(f"WebSocket connection closed with error: {e}")
        
# # === Run with ngrok ===
# from pyngrok import ngrok
# public_url = ngrok.connect(8000)
# print("Assistant public URL:", public_url)
# import nest_asyncio
# nest_asyncio.apply()  # Allows nested event loops in notebooks

# import uvicorn
# from uvicorn.config import Config
# from uvicorn.server import Server

# config = Config(app=app, host="0.0.0.0", port=8000, log_level="info")
# server = Server(config=config)
# await server.serve()


In [ ]:
import numpy as np
import onnxruntime as ort
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoProcessor
import torch
import string
import io
import soundfile as sf
import base64
from fastapi import FastAPI, WebSocket
from starlette.websockets import WebSocketState
import json
import re
import asyncio
import sys
import logging
from huggingface_hub import snapshot_download
from fastapi.middleware.cors import CORSMiddleware
from aiortc import RTCPeerConnection, RTCSessionDescription, AudioStreamTrack
import av

# Logging setup
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
logger = logging.getLogger(__name__)

# Import Spark-TTS
sys.path.append("Spark-TTS")
from sparktts.models.audio_tokenizer import BiCodecTokenizer

# FastAPI app
app = FastAPI()

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Constants
HG_MODEL = "livekit/turn-detector"
ONNX_FILENAME = "/kaggle/input/model_2/onnx/default/1/model_quantized.onnx"
EOU_THRESHOLD = 0.5
MAX_HISTORY = 2
MAX_HISTORY_TOKENS = 512
PUNCS = string.punctuation.replace("'", "")

# Load models
try:
    turn_tokenizer = AutoTokenizer.from_pretrained(HG_MODEL)
    onnx_session = ort.InferenceSession(ONNX_FILENAME, providers=["CPUExecutionProvider"])
    qwen_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")
    qwen_model = AutoModelForCausalLM.from_pretrained(
        "Qwen/Qwen2.5-0.5B-Instruct", torch_dtype=torch.float16
    ).to("cuda")
    snapshot_download("beyoru/Spark-TTS-0.5B-with-KafkaSpark", local_dir="Spark-TTS-0.5B-with-KafkaSpark")
    spark_model = AutoModelForCausalLM.from_pretrained(
        "Spark-TTS-0.5B-with-KafkaSpark/LLM"
    ).to("cuda")
    spark_tokenizer = AutoProcessor.from_pretrained("Spark-TTS-0.5B-with-KafkaSpark/LLM")
    audio_tokenizer = BiCodecTokenizer("Spark-TTS-0.5B-with-KafkaSpark")
except Exception as e:
    logger.error(f"Model loading error: {e}")
    raise

# WebRTC setup
pcs = set()  # Store active peer connections

class AudioStreamTrackImpl(AudioStreamTrack):
    kind = "audio"

    def __init__(self, waveform, sample_rate=16000):
        super().__init__()
        self.waveform = waveform
        self.sample_rate = sample_rate
        self.pos = 0
        self.frame_duration = 0.02  # 20ms frames
        self.samples_per_frame = int(sample_rate * self.frame_duration)

    async def recv(self):
        if self.pos >= len(self.waveform):
            return None
        samples = self.waveform[self.pos:self.pos + self.samples_per_frame]
        self.pos += self.samples_per_frame
        if len(samples) < self.samples_per_frame:
            samples = np.pad(samples, (0, self.samples_per_frame - len(samples)), mode='constant')
        frame = av.AudioFrame.from_ndarray(samples.reshape(-1, 1), format="s16", layout="mono")
        frame.sample_rate = self.sample_rate
        frame.pts = int(self.pos / self.sample_rate * 90000)  # 90kHz clock
        return frame

# Helper functions (unchanged)
def softmax(logits):
    try:
        exp_logits = np.exp(logits - np.max(logits))
        return exp_logits / np.sum(exp_logits)
    except Exception as e:
        logger.error(f"Softmax error: {e}")
        return np.zeros_like(logits)

def normalize_text(text):
    try:
        def strip_puncs(text):
            return text.translate(str.maketrans("", "", PUNCS))
        return " ".join(strip_puncs(text).lower().split())
    except Exception as e:
        logger.error(f"Normalize text error: {e}")
        return text

def format_chat_ctx(chat_ctx):
    try:
        new_chat_ctx = []
        for msg in chat_ctx:
            if msg["role"] in ("user", "assistant"):
                content = normalize_text(msg["content"])
                if content:
                    msg["content"] = content
                    new_chat_ctx.append(msg)
        convo_text = turn_tokenizer.apply_chat_template(
            new_chat_ctx,
            add_generation_prompt=False,
            add_special_tokens=False,
            tokenize=False,
        )
        ix = convo_text.rfind("<|im_end|>")
        return convo_text[:ix]
    except Exception as e:
        logger.error(f"Format chat context error: {e}")
        return ""

def calculate_eou(chat_ctx, session):
    try:
        formatted_text = format_chat_ctx(chat_ctx[-MAX_HISTORY:])
        inputs = turn_tokenizer(
            formatted_text,
            return_tensors="np",
            truncation=True,
            max_length=MAX_HISTORY_TOKENS,
        )
        input_ids = inputs["input_ids"].astype(np.int64)
        outputs = session.run(["logits"], {"input_ids": input_ids})
        logits = outputs[0][0, -1, :]
        probs = softmax(logits)
        eou_token_id = turn_tokenizer.encode("<|im_end|>")[0]
        logger.debug(f"Input text: {formatted_text}, EOU token ID: {eou_token_id}, Probs: {probs}")
        return probs[eou_token_id]
    except Exception as e:
        logger.error(f"EOU calculation error: {e}")
        return 0.0

async def generate_speech_from_text(text: str) -> np.ndarray:
    try:
        prompt = "".join(
            [
                "<|task_tts|>",
                "<|start_content|>",
                text,
                "<|end_content|>",
                "<|start_global_token|>",
            ]
        )
        model_inputs = spark_tokenizer([prompt], return_tensors="pt").to(spark_model.device)
        generated_ids = spark_model.generate(
            **model_inputs,
            max_new_tokens=1024,
            temperature=0.2,
            do_sample=True,
            pad_token_id=spark_tokenizer.pad_token_id,
        )
        generated_ids_trimmed = generated_ids[:, model_inputs.input_ids.shape[1]:]
        predicts_text = spark_tokenizer.batch_decode(
            generated_ids_trimmed, skip_special_tokens=False
        )[0]
        semantic_matches = re.findall(r"<\|bicodec_semantic_(\d+)\|>", predicts_text)
        if not semantic_matches:
            logger.warning("No semantic matches found in TTS output")
            return np.array([], dtype=np.float32)
        pred_semantic_ids = torch.tensor([int(token) for token in semantic_matches]).long().unsqueeze(0)
        global_matches = re.findall(r"<\|bicodec_global_(\d+)\|>", predicts_text)
        pred_global_ids = (
            torch.tensor([int(token) for token in global_matches]).long().unsqueeze(0)
            if global_matches
            else torch.zeros((1, 1), dtype=torch.long)
        )
        pred_global_ids = pred_global_ids.unsqueeze(0)
        wav_np = audio_tokenizer.detokenize(pred_global_ids.squeeze(0), pred_semantic_ids)
        return wav_np
    except Exception as e:
        logger.error(f"Speech generation error: {e}")
        return np.array([], dtype=np.float32)

class ConversationManager:
    def __init__(self):
        self.chat_history = []
        self.sample_rate = 16000
        self.is_processing = False
        self.lock = asyncio.Lock()
        self.transcript_buffer = []

    async def process_conversation(self, transcript: str) -> tuple:
        async with self.lock:
            if self.is_processing:
                logger.info("Skipping transcript processing: already in progress")
                return transcript, 0.0, False, None, None
            self.is_processing = True
            try:
                if not transcript.strip():
                    logger.debug("Received empty transcript, skipping")
                    return transcript, 0.0, False, None, None
                logger.debug(f"Received transcript: {transcript}")
                self.transcript_buffer.append(transcript)
                current_transcript = " ".join(self.transcript_buffer)
                messages = self.chat_history + [{"role": "user", "content": current_transcript}]
                eou_prob = calculate_eou(messages, onnx_session)
                logger.debug(f"EOU score for '{current_transcript}': {eou_prob}")
                is_final = eou_prob >= EOU_THRESHOLD
                if not is_final:
                    logger.debug(f"EOU score {eou_prob} below threshold {EOU_THRESHOLD}, buffering")
                    return current_transcript, eou_prob, False, None, None
                logger.info(f"Processing final transcript: {current_transcript}")
                inputs = qwen_tokenizer.apply_chat_template(
                    messages, add_generation_prompt=True, return_tensors="pt"
                ).to(qwen_model.device)
                outputs = qwen_model.generate(inputs, max_new_tokens=156)
                generated_text = qwen_tokenizer.decode(
                    outputs[0][inputs.shape[1]:], skip_special_tokens=True
                )
                logger.info(f"Generated response: {generated_text}")
                self.chat_history.append({"role": "user", "content": current_transcript})
                self.chat_history.append({"role": "assistant", "content": generated_text})
                if len(self.chat_history) > MAX_HISTORY * 2:
                    self.chat_history = self.chat_history[-MAX_HISTORY * 2:]
                generated_waveform = await generate_speech_from_text(generated_text)
                self.transcript_buffer = []
                logger.debug("Cleared transcript buffer")
                return current_transcript, eou_prob, True, generated_text, generated_waveform
            except Exception as e:
                logger.error(f"Conversation processing error: {e}")
                return current_transcript, eou_prob, False, None, None
            finally:
                self.is_processing = False

async def offer(websocket: WebSocket, waveform: np.ndarray):
    pc = RTCPeerConnection()
    pcs.add(pc)
    track = AudioStreamTrackImpl(waveform)
    pc.addTrack(track)
    await pc.setLocalDescription(await pc.createOffer())
    await websocket.send_json({"sdp": pc.localDescription.sdp, "type": pc.localDescription.type})
    answer = json.loads(await websocket.receive_text())
    await pc.setRemoteDescription(RTCSessionDescription(sdp=answer["sdp"], type=answer["type"]))
    return pc

@app.get("/")
async def root():
    return {"message": "Assistant WebSocket server is running. Connect via /ws/assistant"}

@app.get("/test")
async def test_page():
    return """
    <!DOCTYPE html>
    <html>
    <head><title>Assistant WS Test</title></head>
    <body>
    <h1>Assistant WebSocket Test</h1>
    <textarea id="transcript" rows="4" cols="50"></textarea><br/>
    <button onclick="sendTranscript()">Send Transcript</button>
    <pre id="output"></pre>
    <script>
      const ws = new WebSocket("ws://" + location.host + "/ws/assistant");
      ws.onmessage = (event) => {
        const data = JSON.parse(event.data);
        document.getElementById('output').textContent += JSON.stringify(data, null, 2) + "\\n";
      };
      function sendTranscript() {
        const transcript = document.getElementById('transcript').value;
        ws.send(JSON.stringify({transcript: transcript}));
      }
    </script>
    </body>
    </html>
    """

@app.on_event("startup")
async def startup_event():
    logger.info("Starting Assistant server...")

@app.on_event("shutdown")
async def shutdown_event():
    logger.info("Shutting down Assistant server...")

@app.websocket("/ws/assistant")
async def websocket_endpoint(websocket: WebSocket):
    await websocket.accept()
    cm = ConversationManager()
    try:
        while True:
            try:
                data = json.loads(await websocket.receive_text())
                if "sdp" in data and "type" in data:
                    pc = RTCPeerConnection()
                    pcs.add(pc)
                    await pc.setRemoteDescription(RTCSessionDescription(sdp=data["sdp"], type=data["type"]))
                    track = AudioStreamTrackImpl(np.array([]))  # Placeholder
                    pc.addTrack(track)
                    await pc.setLocalDescription(await pc.createAnswer())
                    await websocket.send_json({"sdp": pc.localDescription.sdp, "type": pc.localDescription.type})
                    continue
                transcript = data.get("transcript", "")
                logger.info(f"WebSocket input: transcript={transcript}")
                if not transcript.strip():
                    continue
                transcript, eou_score, complete, resp, waveform = await cm.process_conversation(transcript)
                await websocket.send_json({
                    "type": "transcript",
                    "transcript": transcript,
                    "eou_score": float(eou_score),
                    "is_complete": complete,
                })
                if resp and waveform is not None:
                    await asyncio.sleep(0.1)
                    await websocket.send_json({"type": "assistant_response", "response": resp})
                    await offer(websocket, waveform)
            except Exception as e:
                logger.error(f"WebSocket error: {e}")
                if websocket.client_state == WebSocketState.CONNECTED:
                    await websocket.close(code=1000, reason=str(e))
    except Exception as e:
        logger.error(f"WebSocket connection closed with error: {e}")
    finally:
        for pc in pcs:
            await pc.close()
        pcs.clear()

# === Run with ngrok ===
from pyngrok import ngrok
public_url = ngrok.connect(8000)
print("Assistant public URL:", public_url)
import nest_asyncio
nest_asyncio.apply()

import uvicorn
from uvicorn.config import Config
from uvicorn.server import Server

config = Config(app=app, host="0.0.0.0", port=8000, log_level="info")
server = Server(config=config)
await server.serve()

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Fetching 32 files:   0%|          | 0/32 [00:00<?, ?it/s]

config.yaml:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Missing tensor: mel_transformer.spectrogram.window
Missing tensor: mel_transformer.mel_scale.fb


/tmp/ipykernel_35/2975910188.py:285: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")
/tmp/ipykernel_35/2975910188.py:289: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("shutdown")


Assistant public URL: NgrokTunnel: "https://03d7-34-30-116-120.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [35]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     ('2405:4803:fe20:bb70:1c0f:707b:23f4:8799', 0) - "WebSocket /ws/assistant" [accepted]
INFO:     connection open
INFO:     ('2405:4803:fe20:bb70:1c0f:707b:23f4:8799', 0) - "WebSocket /ws/assistant" [accepted]
INFO:     connection open
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
